## 載入套件

In [1]:
import requests, bs4
from datetime import datetime
import random, time
import pandas as pd

## 設定函數

In [2]:
def get_data(URL):
    
    #my_headers = {'cookie': 'over18=1;'} 
    my_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',"Connection": "close"}
    response = requests.get(URL, headers = my_headers)
    #response = requests.get(URL)
    
    soup = bs4.BeautifulSoup(response.text,"html.parser")
    
    header = soup.find_all('span','article-meta-value')
    
    author = header[0].text
    board = header[1].text
    title = header[2].text
    date = header[3].text
    
    
    main_container = soup.find(id='main-container')
    all_text = main_container.text
    pre_text = all_text.split('--')[0]
    texts = pre_text.split('\n')
    contents = texts[2:]

    content = '\n'.join(contents)
    
    # print('作者：'+author)
    # print('看板：'+board)
    # print('標題：'+title)
    # print('日期：'+date)
    # print('內容：'+content)
    
    return [author, board, title, date, content]

In [3]:
def get_all_URL(URL):
    r = requests.get(URL)
    soup = bs4.BeautifulSoup(r.text,"html.parser")
    articles = soup.find_all('div','title')
    
    dates = soup.find_all('div','date')
    page_last_date = dates[-6].text.strip()
    
    hrefs = []
    for item in articles:
        a_item = item.select_one("a")
        if a_item:
            item_href = 'https://www.ptt.cc'+a_item.get('href')
            hrefs.append(item_href)
            
    return (hrefs, page_last_date)

## 抓取每篇文章連結

In [9]:
url = "https://www.ptt.cc/bbs/stock/index.html"

all_artical_hrefs = []

start_year = '2021'
latest_date = datetime.strptime('2021/5/20','%Y/%m/%d')
page_last_date = datetime.strptime(start_year+'/5/30','%Y/%m/%d')
up_page_date = datetime.strptime(start_year+'/5/30','%Y/%m/%d')

error_page = []

while page_last_date >= latest_date:
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text,"html.parser")
    
    # try to get URLs
    try:
        hrefs, date = get_all_URL(url)
    except:
        hrefs = []
        error_page.append(url)
    
    # save href and transform time
    all_artical_hrefs += hrefs
    page_last_date = datetime.strptime('1904/'+ date,'%Y/%m/%d') ## avoiding 2/29 error
    
    # change year (optional)
    if up_page_date.month == 1 and page_last_date.month == 12:
        start_year = str(int(start_year)-1)
    
    # print date and record this date
    page_last_date = datetime.strptime(start_year+'/'+date,'%Y/%m/%d')
    if page_last_date != up_page_date:
        print(str(page_last_date))
    up_page_date = page_last_date
    
    # find next_page_url
    btn = soup.select('div.btn-group > a')
    up_page_href = btn[3]['href']
    next_page_url = 'https://www.ptt.cc' + up_page_href
    url = next_page_url

2021-06-02 00:00:00
2021-06-01 00:00:00
2021-05-31 00:00:00
2021-05-30 00:00:00
2021-05-29 00:00:00
2021-05-28 00:00:00
2021-05-27 00:00:00
2021-05-26 00:00:00
2021-05-25 00:00:00
2021-05-24 00:00:00
2021-05-23 00:00:00
2021-05-22 00:00:00
2021-05-21 00:00:00
2021-05-20 00:00:00
2021-05-19 00:00:00


In [10]:
len(all_artical_hrefs)
artical_herfs_pd = pd.DataFrame(all_artical_hrefs, columns = ['link'])
artical_herfs_pd.to_csv('PTT_stock_links_part2.csv',index = False)

半途讀檔

In [9]:
hrefs = pd.read_csv('PTT_nCoV2019_links.csv')
all_artical_hrefs = hrefs['link'].to_list()
len(all_artical_hrefs)

47131

## 抓取每篇文章內容

In [11]:
data = []
error_articals = []

ct = 0

for URL in all_artical_hrefs[:]:
    print('Try to connect...')
    try:
        data.append(get_data(URL)+[URL])
        ct += 1
        print("{} articals has been recorder".format(ct))
    except:
        print('Something Error...')
        error_articals.append(URL)
        continue
        
    sec = random.random()
    print('Sleeping for {} seconds...'.format(round(sec,2)))
    time.sleep(sec)
    
PTT_data_slice = pd.DataFrame(data, columns = ['Author','board','title','date','content','url'])


Try to connect...
1 articals has been recorder
Sleeping for 0.15 seconds...
Try to connect...
2 articals has been recorder
Sleeping for 0.13 seconds...
Try to connect...
3 articals has been recorder
Sleeping for 0.28 seconds...
Try to connect...
4 articals has been recorder
Sleeping for 0.47 seconds...
Try to connect...
5 articals has been recorder
Sleeping for 0.24 seconds...
Try to connect...
6 articals has been recorder
Sleeping for 0.52 seconds...
Try to connect...
7 articals has been recorder
Sleeping for 0.08 seconds...
Try to connect...
8 articals has been recorder
Sleeping for 0.14 seconds...
Try to connect...
9 articals has been recorder
Sleeping for 0.86 seconds...
Try to connect...
10 articals has been recorder
Sleeping for 0.01 seconds...
Try to connect...
11 articals has been recorder
Sleeping for 0.48 seconds...
Try to connect...
12 articals has been recorder
Sleeping for 0.72 seconds...
Try to connect...
13 articals has been recorder
Sleeping for 0.22 seconds...
Try to c

107 articals has been recorder
Sleeping for 0.26 seconds...
Try to connect...
108 articals has been recorder
Sleeping for 0.74 seconds...
Try to connect...
109 articals has been recorder
Sleeping for 0.25 seconds...
Try to connect...
110 articals has been recorder
Sleeping for 0.78 seconds...
Try to connect...
111 articals has been recorder
Sleeping for 0.65 seconds...
Try to connect...
112 articals has been recorder
Sleeping for 0.17 seconds...
Try to connect...
113 articals has been recorder
Sleeping for 0.42 seconds...
Try to connect...
114 articals has been recorder
Sleeping for 0.59 seconds...
Try to connect...
115 articals has been recorder
Sleeping for 0.38 seconds...
Try to connect...
116 articals has been recorder
Sleeping for 0.38 seconds...
Try to connect...
117 articals has been recorder
Sleeping for 0.07 seconds...
Try to connect...
118 articals has been recorder
Sleeping for 0.77 seconds...
Try to connect...
119 articals has been recorder
Sleeping for 0.46 seconds...
Try 

212 articals has been recorder
Sleeping for 0.54 seconds...
Try to connect...
213 articals has been recorder
Sleeping for 0.58 seconds...
Try to connect...
214 articals has been recorder
Sleeping for 0.19 seconds...
Try to connect...
215 articals has been recorder
Sleeping for 0.12 seconds...
Try to connect...
216 articals has been recorder
Sleeping for 1.0 seconds...
Try to connect...
217 articals has been recorder
Sleeping for 0.06 seconds...
Try to connect...
218 articals has been recorder
Sleeping for 0.2 seconds...
Try to connect...
219 articals has been recorder
Sleeping for 0.19 seconds...
Try to connect...
220 articals has been recorder
Sleeping for 0.37 seconds...
Try to connect...
221 articals has been recorder
Sleeping for 0.56 seconds...
Try to connect...
222 articals has been recorder
Sleeping for 0.1 seconds...
Try to connect...
223 articals has been recorder
Sleeping for 0.23 seconds...
Try to connect...
224 articals has been recorder
Sleeping for 0.8 seconds...
Try to c

Try to connect...
318 articals has been recorder
Sleeping for 0.19 seconds...
Try to connect...
319 articals has been recorder
Sleeping for 0.49 seconds...
Try to connect...
320 articals has been recorder
Sleeping for 0.84 seconds...
Try to connect...
321 articals has been recorder
Sleeping for 0.68 seconds...
Try to connect...
322 articals has been recorder
Sleeping for 0.75 seconds...
Try to connect...
323 articals has been recorder
Sleeping for 0.61 seconds...
Try to connect...
324 articals has been recorder
Sleeping for 0.94 seconds...
Try to connect...
325 articals has been recorder
Sleeping for 0.9 seconds...
Try to connect...
326 articals has been recorder
Sleeping for 0.63 seconds...
Try to connect...
327 articals has been recorder
Sleeping for 0.81 seconds...
Try to connect...
328 articals has been recorder
Sleeping for 0.89 seconds...
Try to connect...
329 articals has been recorder
Sleeping for 0.58 seconds...
Try to connect...
330 articals has been recorder
Sleeping for 0.0

423 articals has been recorder
Sleeping for 0.26 seconds...
Try to connect...
424 articals has been recorder
Sleeping for 0.7 seconds...
Try to connect...
425 articals has been recorder
Sleeping for 0.42 seconds...
Try to connect...
426 articals has been recorder
Sleeping for 0.51 seconds...
Try to connect...
427 articals has been recorder
Sleeping for 0.4 seconds...
Try to connect...
428 articals has been recorder
Sleeping for 0.16 seconds...
Try to connect...
429 articals has been recorder
Sleeping for 0.5 seconds...
Try to connect...
430 articals has been recorder
Sleeping for 0.16 seconds...
Try to connect...
431 articals has been recorder
Sleeping for 0.06 seconds...
Try to connect...
432 articals has been recorder
Sleeping for 0.02 seconds...
Try to connect...
433 articals has been recorder
Sleeping for 0.56 seconds...
Try to connect...
434 articals has been recorder
Sleeping for 0.2 seconds...
Try to connect...
435 articals has been recorder
Sleeping for 0.43 seconds...
Try to c

...

1366 articals has been recorder
Sleeping for 0.81 seconds...
Try to connect...
1367 articals has been recorder
Sleeping for 0.2 seconds...
Try to connect...
1368 articals has been recorder
Sleeping for 0.63 seconds...
Try to connect...
1369 articals has been recorder
Sleeping for 0.04 seconds...
Try to connect...
1370 articals has been recorder
Sleeping for 0.19 seconds...
Try to connect...
1371 articals has been recorder
Sleeping for 0.35 seconds...
Try to connect...
1372 articals has been recorder
Sleeping for 0.53 seconds...
Try to connect...
1373 articals has been recorder
Sleeping for 0.87 seconds...
Try to connect...
1374 articals has been recorder
Sleeping for 0.13 seconds...
Try to connect...
1375 articals has been recorder
Sleeping for 0.41 seconds...
Try to connect...
1376 articals has been recorder
Sleeping for 0.77 seconds...
Try to connect...
1377 articals has been recorder
Sleeping for 0.64 seconds...
Try to connect...
1378 articals has been recorder
Sleeping for 0.42 sec

1470 articals has been recorder
Sleeping for 0.44 seconds...
Try to connect...
1471 articals has been recorder
Sleeping for 0.21 seconds...
Try to connect...
1472 articals has been recorder
Sleeping for 0.66 seconds...
Try to connect...
1473 articals has been recorder
Sleeping for 0.43 seconds...
Try to connect...
1474 articals has been recorder
Sleeping for 0.03 seconds...
Try to connect...
1475 articals has been recorder
Sleeping for 0.95 seconds...
Try to connect...
1476 articals has been recorder
Sleeping for 0.52 seconds...
Try to connect...
1477 articals has been recorder
Sleeping for 0.73 seconds...
Try to connect...
1478 articals has been recorder
Sleeping for 0.33 seconds...
Try to connect...
1479 articals has been recorder
Sleeping for 0.66 seconds...
Try to connect...
1480 articals has been recorder
Sleeping for 0.41 seconds...
Try to connect...
1481 articals has been recorder
Sleeping for 0.82 seconds...
Try to connect...
1482 articals has been recorder
Sleeping for 0.84 se

1574 articals has been recorder
Sleeping for 0.73 seconds...
Try to connect...
1575 articals has been recorder
Sleeping for 0.77 seconds...
Try to connect...
1576 articals has been recorder
Sleeping for 0.05 seconds...
Try to connect...
1577 articals has been recorder
Sleeping for 0.37 seconds...
Try to connect...
1578 articals has been recorder
Sleeping for 0.21 seconds...
Try to connect...
1579 articals has been recorder
Sleeping for 0.38 seconds...
Try to connect...
1580 articals has been recorder
Sleeping for 0.02 seconds...
Try to connect...
1581 articals has been recorder
Sleeping for 0.11 seconds...
Try to connect...
1582 articals has been recorder
Sleeping for 0.08 seconds...
Try to connect...
1583 articals has been recorder
Sleeping for 0.68 seconds...
Try to connect...
1584 articals has been recorder
Sleeping for 0.77 seconds...
Try to connect...
1585 articals has been recorder
Sleeping for 0.77 seconds...
Try to connect...
1586 articals has been recorder
Sleeping for 0.01 se

1678 articals has been recorder
Sleeping for 0.55 seconds...
Try to connect...
1679 articals has been recorder
Sleeping for 0.51 seconds...
Try to connect...
1680 articals has been recorder
Sleeping for 0.92 seconds...
Try to connect...
1681 articals has been recorder
Sleeping for 0.65 seconds...
Try to connect...
1682 articals has been recorder
Sleeping for 0.55 seconds...
Try to connect...
1683 articals has been recorder
Sleeping for 0.66 seconds...
Try to connect...
1684 articals has been recorder
Sleeping for 0.97 seconds...
Try to connect...
1685 articals has been recorder
Sleeping for 0.33 seconds...
Try to connect...
1686 articals has been recorder
Sleeping for 0.3 seconds...
Try to connect...
1687 articals has been recorder
Sleeping for 0.44 seconds...
Try to connect...
1688 articals has been recorder
Sleeping for 0.92 seconds...
Try to connect...
1689 articals has been recorder
Sleeping for 0.96 seconds...
Try to connect...
1690 articals has been recorder
Sleeping for 0.35 sec

1782 articals has been recorder
Sleeping for 0.04 seconds...
Try to connect...
1783 articals has been recorder
Sleeping for 0.82 seconds...
Try to connect...
1784 articals has been recorder
Sleeping for 0.93 seconds...
Try to connect...
1785 articals has been recorder
Sleeping for 0.47 seconds...
Try to connect...
1786 articals has been recorder
Sleeping for 0.11 seconds...
Try to connect...
1787 articals has been recorder
Sleeping for 0.75 seconds...
Try to connect...
1788 articals has been recorder
Sleeping for 0.77 seconds...
Try to connect...
1789 articals has been recorder
Sleeping for 0.16 seconds...
Try to connect...
1790 articals has been recorder
Sleeping for 0.23 seconds...
Try to connect...
1791 articals has been recorder
Sleeping for 0.3 seconds...
Try to connect...
1792 articals has been recorder
Sleeping for 0.76 seconds...
Try to connect...
1793 articals has been recorder
Sleeping for 0.84 seconds...
Try to connect...
1794 articals has been recorder
Sleeping for 0.11 sec

In [12]:
#PTT_data = pd.DataFrame()
#PTT_data = pd.concat([PTT_data, PTT_data_slice], axis = 0)
PTT_data_slice 

,Author,board,title,date,content,url
0,zxcvxx (zxcvxx),Stock,[新聞] 日本及豐田可能在固態電池技術領先嗎？但,Wed Jun 2 17:04:07 2021,原文標題：\n日本及豐田可能在固態電池技術領先嗎？但量產布局過於保守將成為致命傷\n ...,https://www.ptt.cc/bbs/Stock/M.1622624653.A.30...
1,bmw606042001 (多空雙buff),Stock,[新聞] 貨櫃三雄弱了 航運股「這族群」全拉漲停,Wed Jun 2 17:08:32 2021,\n原文標題：\n貨櫃三雄弱了 航運股「這族群」全拉漲停\n原文連結：\nhttps://c...,https://www.ptt.cc/bbs/Stock/M.1622624914.A.2B...
2,kiversonx17 (閉著眼睛買玉山金),Stock,[新聞] 金管會澄清 公股買高端是客戶下單非投資,Wed Jun 2 17:09:59 2021,\n原文標題：金管會幫澄清 公股買高端是客戶下單非投資\n ...,https://www.ptt.cc/bbs/Stock/M.1622625001.A.42...
3,QQmmmmmm (數大便是美),Stock,Re: [請益] 手上有高端疫苗 心情怎麼調適,Wed Jun 2 17:21:49 2021,\n\n郭董想進口疫苗恐遇法律難關 美教授直指最關鍵問題\n\n〔即時新聞／綜合報導〕鴻海創...,https://www.ptt.cc/bbs/Stock/M.1622625713.A.A1...
4,ljsnonocat2 (凡所有相皆是虛妄),Stock,[新聞] 皇翔處分長榮511.3萬股，獲利約2227萬元,Wed Jun 2 17:24:50 2021,原文標題：\n皇翔5/27~6/1處分長榮511.3萬股，獲利約2227萬元\n\n原文連結...,https://www.ptt.cc/bbs/Stock/M.1622625893.A.47...
...,...,...,...,...,...,...
1854,justin1943 (prototype),Stock,Re: [新聞] 加密貨幣崩潰加速 美股、債券均在下跌,Wed May 19 22:17:38 2021,\n明天慘了\n\n這次虛擬幣大跌不像上次\n這次很多機構的portfolio 都有一定比例...,https://www.ptt.cc/bbs/Stock/M.1621433860.A.49...
1855,VIIBlack (Black No. VII),Stock,[情報] 血洗幣圈！加密貨幣交易所Coinbase和Bina,Wed May 19 22:19:30 2021,血洗幣圈！加密貨幣交易所Coinbase和Binance故障中斷\nhttps://news...,https://www.ptt.cc/bbs/Stock/M.1621433973.A.8F...
1856,golang (Gopher),Stock,Re: [新聞] 加密貨幣崩潰加速 美股、債券均在下跌,Wed May 19 22:24:53 2021,※ 引述《justin1943 (prototype)》之銘言：\n: 明天慘了\n: 這次...,https://www.ptt.cc/bbs/Stock/M.1621434294.A.F9...
1857,ARKUE (),Stock,Re: [心得] 勸世文,Wed May 19 22:34:39 2021,不知道為什麼 讓我想到2015的這一篇\n\nhttps://i.imgur.com/VAt...,https://www.ptt.cc/bbs/Stock/M.1621434881.A.9A...


## 存取檔案

In [13]:
import pandas as pd

#PTT_data = pd.DataFrame(data, columns = ['Author','board','title','date','content','url'])
PTT_data_slice.to_csv('ptt_stock_new.csv')